In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb rank_bm25 faiss-cpu pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
# importation des librairies
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from pprint import pprint
import os
from getpass import getpass
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
# configuration du mot de passe en variable d'environnement
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [ ]:
# mis en place de l'enbedding
Embedding = OpenAIEmbeddings()

In [ ]:
# creation d'une liste pour test
doc_list = [
    "J'apprécie les fruits frais comme les pommes et les poires",
    "Les agrumes comme les oranges et les citrons sont riches en vitamine C",
    "La technologie d'Apple, telle que les iPhone et les MacBook, est très populaire",
    "Je préfère boire du jus d'orange frais le matin",
    "Les salades de fruits sont délicieuses pendant l'été",
    "Le système d'exploitation macOS est développé par Apple",
    "Les smoothies aux fruits sont à la fois nutritifs et rafraîchissants",
    "Mon fruit préféré est la mangue",
    "Je vais au marché pour acheter des fruits biologiques",
    "Les ordinateurs Apple sont souvent utilisés par les professionnels créatifs"
]


In [ ]:
# BM25 est souvent utilisé dans les systèmes de recherche d'information et les moteurs de recherche pour évaluer la pertinence d'un document par rapport à une requête de l'utilisateur.
bm25_retriver = BM25Retriever.from_texts(doc_list)
bm25_retriver.k=2

In [ ]:

vectorstore = Chroma.from_texts(doc_list,embedding=Embedding)

In [ ]:

retriver_vector = vectorstore.as_retriever(search_kwargs={"k":2})

In [ ]:
bm25_retriver.get_relevant_documents("chercher une pomme pour faire un desert")

[Document(page_content='Je vais au marché pour acheter des fruits biologiques'),
 Document(page_content='Les ordinateurs Apple sont souvent utilisés par les professionnels créatifs')]

In [ ]:
retriver_vector.get_relevant_documents(" achete un nouveau PC")

[Document(page_content='Les ordinateurs Apple sont souvent utilisés par les professionnels créatifs'),
 Document(page_content='Les ordinateurs Apple sont souvent utilisés par les professionnels créatifs')]

In [ ]:
ensemble_retriver = EnsembleRetriever(retrievers=[retriver_vector,bm25_retriver],weights=[0.5,0.5])

In [ ]:
ensemble_retriver.get_relevant_documents("la radio")

[Document(page_content="La technologie d'Apple, telle que les iPhone et les MacBook, est très populaire"),
 Document(page_content='Mon fruit préféré est la mangue'),
 Document(page_content='Les smoothies aux fruits sont à la fois nutritifs et rafraîchissants')]

In [1]:
## Guide des aides à la Renovation
#https://france-renov.gouv.fr/guide-aides-financieres-2024

In [ ]:
path = "/content/Guide des aides rénovation 2024 (1).pdf"
from pypdf import PdfReader

In [ ]:
doc_reader = PdfReader(path)

In [ ]:
## load the document
loader= PyPDFLoader(path)
data = loader.load()

In [ ]:
# Splitting up the text into smaller chunks for indexing
text_spliter = RecursiveCharacterTextSplitter(separators="\n", chunk_size=1000,chunk_overlap=200,length_function= len)
text = text_spliter.split_documents(data)

In [ ]:
len(text)

169

In [ ]:
# intialisation
bm25_retriver = BM25Retriever.from_documents(text)
bm25_retriver.k = 2

In [ ]:
bm25_retriver.get_relevant_documents("montant de l'aide pour de la pompe à chaleur air/eau")

[Document(page_content='sification européenne).Pour obtenir une aide liée au dispositif des \nCEE, le COP doit être supérieur à 2,5 pour \nune installation sur air extrait et 2,4 dans les \nautres cas.\nDans le cas d’une pompe à chaleur air/eau \ncomportant un dispositif d’appoint utilisant \nun combustible liquide ou gazeux et une \nrégulation qui les pilote, le taux de couverture \nde la pompe à chaleur hors dispositif d’ap -\npoint est supérieur ou égal à 70 %. \nLes chauffe-eau et le chauffage solaire \nLes équipements de chauffage ou d’eau \nchaude sanitaire fonctionnant à l’énergie \nsolaire doivent être dotés de capteurs solaires \ndisposant d’une certification CSTBat ou de la \ncertification Solar Keymark ou équivalente. \nLes capteurs peuvent être thermiques (à air \nou à circulation d’eau ou d’eau glycolée) ou \nhybrides thermiques et électriques à circu -\nlation d’eau ou d’eau glycolée. Les critères \nde performance de l’équipement (valeurs à \nrespecter indiquées ci-dessou

In [ ]:
vector_store = Chroma.from_documents(data,embedding=Embedding)

In [ ]:
from re import search
retriver_vector = vector_store.as_retriever(search_kwargs={"k":2})

In [ ]:
retriver_vector.get_relevant_documents("montant de l'aide pour de la pompe à chaleur air/eau")

[Document(page_content='PLAFONNEMENT DES DÉPENSES ÉLIGIBLES \n La dépense éligible correspond au coût du matériel, pose comprise. Le montant ne tient pas compte des remises, \nristournes ou rabais proposés par les entreprises. \nÉQUIPEMENTS ET MATÉRIAUX ÉLIGIBLESPLAFOND DE DÉPENSE \nÉLIGIBLE*\nCHAUFFAGE ET EAU CHAUDE SANITAIRE\nRaccordement à un réseau de chaleur et/ou de froid 1 800 €\nChauffe-eau thermodynamique 3 500 €\nPompe à chaleur air/eau (dont PAC hybrides) 12 000 €\nPompe à chaleur géothermique ou solarothermique (dont PAC hybrides) 18 000 €\nChauffe-eau solaire individuel et dispositifs solaires pour \nle chauffage de l’eau (dont appoint)7 000 €\nSystème solaire combiné et dispositifs solaires pour \nle chauffage des locaux (dont appoint) en Métropole16 000 €\nPartie thermique d’un équipement PVT eau  \n(système hybride photovoltaïque et thermique)4 000 €\nPoêle à bûches et cuisinière à bûches 4 000 €\nPoêle à granulés et cuisinière à granulés 5 000 €\nChaudière bois à alime

In [ ]:
# ensemble retriver
ensemble_retriver = EnsembleRetriever(retrievers=[retriver_vector,bm25_retriver],weights=[0.5,0.5])

In [ ]:
ensemble_retriver.get_relevant_documents("montant de l'aide pour de la pompe à chaleur air/eau")

[Document(page_content='PLAFONNEMENT DES DÉPENSES ÉLIGIBLES \n La dépense éligible correspond au coût du matériel, pose comprise. Le montant ne tient pas compte des remises, \nristournes ou rabais proposés par les entreprises. \nÉQUIPEMENTS ET MATÉRIAUX ÉLIGIBLESPLAFOND DE DÉPENSE \nÉLIGIBLE*\nCHAUFFAGE ET EAU CHAUDE SANITAIRE\nRaccordement à un réseau de chaleur et/ou de froid 1 800 €\nChauffe-eau thermodynamique 3 500 €\nPompe à chaleur air/eau (dont PAC hybrides) 12 000 €\nPompe à chaleur géothermique ou solarothermique (dont PAC hybrides) 18 000 €\nChauffe-eau solaire individuel et dispositifs solaires pour \nle chauffage de l’eau (dont appoint)7 000 €\nSystème solaire combiné et dispositifs solaires pour \nle chauffage des locaux (dont appoint) en Métropole16 000 €\nPartie thermique d’un équipement PVT eau  \n(système hybride photovoltaïque et thermique)4 000 €\nPoêle à bûches et cuisinière à bûches 4 000 €\nPoêle à granulés et cuisinière à granulés 5 000 €\nChaudière bois à alime

In [ ]:

from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm = ChatOpenAI(model="gpt-4",max_tokens=300,temperature=0.2)

In [ ]:
renovation_template = """
Vous êtes un spécialiste des aides à la rénovation qui s'appuie sur des connaissances approfondies des réglementations et des programmes d'assistance en France.
Utilisez le contexte fourni comme base pour vos réponses et ne déviez pas vers de nouveaux chemins de raisonnement -
contentez-vous d'associer ce que vous savez déjà.
Vos réponses doivent être concises et précises, et se limiter exclusivement aux sujets des aides à la rénovation et des dispositifs associés tels que ceux proposés par l'ANAH.

Donne egalement la source de tes réponses, la page de tes sources exemple:
quel sont les aides pour installer une pompe a chaleur air/eau ?
Le montant des aides pour l'installation d'une pompe à chaleur air/eau dépend des ressources du ménage. Pour les ménages aux ressources très modestes, l'aide est de 5 000 €. Pour les ménages aux ressources modestes, l'aide est de 4 000 €. Pour les ménages aux ressources intermédiaires, l'aide est de 3 000 €. Les ménages aux ressources supérieures ne sont pas éligibles à cette aide.


Source : Document 1, page 14, Guide des aides rénovation 2024.

CONTEXTE :
{context}

QUESTION : {question}

VOTRE RÉPONSE :"""

renovation_prompt = ChatPromptTemplate.from_template(renovation_template)


chain = (
    {"context": ensemble_retriver, "question": RunnablePassthrough()}
    | renovation_prompt
    | llm
    | StrOutputParser()
)


In [ ]:
print(chain.invoke("quelles sont les aides pour installer une pompe a chaleur eau/eau"))

Le montant des aides pour l'installation d'une pompe à chaleur eau/eau dépend des ressources du ménage. Pour les ménages aux ressources très modestes, l'aide est de 5 000 €. Pour les ménages aux ressources modestes, l'aide est de 4 000 €. Pour les ménages aux ressources intermédiaires, l'aide est de 3 000 €. Les ménages aux ressources supérieures ne sont pas éligibles à cette aide.

Source : Document 1, page 48, Guide des aides rénovation 2024.


In [ ]:
print(chain.invoke("quelles sont les conditions a respecter pour beneficier des aides pour une pompe à chaleur air/eau"))

Pour bénéficier des aides pour une pompe à chaleur air/eau, plusieurs conditions doivent être respectées :

1. La pompe à chaleur doit avoir une efficacité énergétique saisonnière pour le chauffage, calculée avec son appoint électrique ou à combustible fossile, supérieure ou égale à 126% si elle fonctionne à basse température, ou supérieure ou égale à 111% si elle fonctionne à moyenne et haute température.

2. Si la pompe à chaleur air/eau comporte un dispositif d'appoint utilisant un combustible liquide ou gazeux et une régulation qui les pilote, le taux de couverture de la pompe à chaleur hors dispositif d'appoint doit être supérieur ou égal à 70%.

3. Les travaux doivent permettre d'améliorer la performance énergétique du logement et respecter des exigences de performances minimales.

4. Les travaux doivent être réalisés par un professionnel RGE (Reconnu Garant de l'Environnement).

Sources : Document 1, page 48, Guide des aides rénovation 2024 ; Document 3, page 27, Guide des aides

In [ ]:
print(chain.invoke("quel est le plafond de dépenses éligible pour installer un réseau de chaleur"))

Le plafond de dépenses éligibles pour l'installation d'un réseau de chaleur est de 1 800 €.

Source : Document 1, page 16, Guide des aides rénovation 2024.


In [ ]:
# ALLUCINATION DU MODEL
print(chain.invoke(" l'installation d'une chaudière au gaz peut il beneficer des aides de l'ANAH "))

Oui, l'installation d'une chaudière au gaz peut bénéficier des aides de l'ANAH, à condition que celle-ci remplace une chaudière au charbon, au fioul ou au gaz autre qu'à condensation. Le nouvel équipement doit utiliser des énergies renouvelables. Les travaux doivent être réalisés par un professionnel RGE et respecter des exigences de performances minimales.

Source : Document 1, page 27, Guide des aides rénovation 2024.


In [ ]:
# la bonne réponse dépend de la facon dont la question est posée
print(chain.invoke(" les chaudieres au gaz qui utlisent donc une energie non renouvelables sont elles exclues des aides "))

Oui, les chaudières au gaz qui utilisent une énergie non renouvelable sont exclues des aides. Les aides sont destinées au remplacement d'une chaudière au charbon, au fioul ou au gaz (autres qu'à condensation) par un équipement utilisant des énergies renouvelables.

Source : Document 3, page 27, Guide des aides rénovation 2024.


In [ ]:
print(chain.invoke("comment contacter un conseiller France Renov"))

Pour contacter un conseiller France Rénov', vous pouvez vous rendre sur le site internet france-renov.gouv.fr ou appeler le numéro 0 808 800 700.

Source : Document 2, page 61, Guide des aides rénovation 2024.


plus de ressources:

https://www.youtube.com/watch?v=dUkiQ_WI92c&t=807s

https://www.youtube.com/watch?v=lYxGYXjfrNI

https://python.langchain.com/docs/integrations/vectorstores/astradb

https://france-renov.gouv.fr/guide-aides-financieres-2024